In [1]:
# using methods from 04, now loop through every link, build large dataframe and save it

In [2]:
# Import Packages
import time, random
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# Selenium setup
import os
from selenium import webdriver
chromedriver="/Applications/chromedriver"
os.environ["webdriver.chrome.driver"]=chromedriver
driver=webdriver.Chrome(chromedriver)

/var/folders/tj/465rg90j3m59mm_rhyq5qcx40000gn/T/ipykernel_4579/1436372225.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromedriver)


In [3]:
# SQL setup
from sqlalchemy import create_engine
# connect to a local database
engine = create_engine("sqlite:///boxscores.db")

In [4]:
# Import saved pickle dataframe from last save point
# Or start with empty dataframe from scratch

#main_df=pd.DataFrame()
main_df=pd.read_pickle('saved_main_df.pkl')

In [11]:
#Loading in the links to be scraped

links=[]
# open file and read the content in a list
with open(r'boxscore_links.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        links.append(x)

In [6]:
# loop thorugh every link
for link in links[2235:]:
    
    print('Link: '+link)
    
    url=link
    
    # get html
    time.sleep(1+2*random.random())
    driver.get(url)
    time.sleep(1+2*random.random())
    html=driver.page_source
    soup=bs(html)
    
    # Grab the entire scorebox
    scorebox=soup.find('table',{'class':'linescore nohover stats_table no_freeze'}).find_all('tr')
    
    # Separate the three rows of interest
    scorebox_row1=scorebox[0].find_all('th')
    scorebox_row2=scorebox[1].find_all('td')
    scorebox_row3=scorebox[2].find_all('td')
    
    # Create dataframe that will be the scorebox
    # set first column name to Team
    scorebox_column_names=['Team']

    # Grab column names and add to the list
    for tag in scorebox_row1[2:]:
        scorebox_column_names.append(tag.text)
        
    # Grab values of row 2
    row2_values=[]
    for tag in scorebox_row2[1:]:
        row2_values.append(tag.text)
        
    # Grab values of row 3
    row3_values=[]
    for tag in scorebox_row3[1:]:
        row3_values.append(tag.text)
        
    # Make rows as dictionaries
    row1=dict(list(zip(scorebox_column_names,row2_values)))
    row2=dict(list(zip(scorebox_column_names,row3_values)))

    # Save as list of dictionaries
    rows=[row1,row2]
    
    # Create the scorebox dataframe using the row dictionaries list
    scorebox_df=pd.DataFrame(rows)
    
    # Defining team1 and team2
    team1=scorebox_df['Team'][0]
    team2=scorebox_df['Team'][1]
    
    print(team1+' vs '+team2)
    
    # Now create team batting and pitching dataframes
    # This finds the 4 batting and pitching tables (2 per team)
    batting_pitching_tables=soup.find_all('table',{'class':'sortable stats_table min_width shade_zero now_sortable'})
     
    # Separate the batting and pitching tables
    batting_tables=[batting_pitching_tables[0],batting_pitching_tables[1]]
    pitching_tables=[batting_pitching_tables[2],batting_pitching_tables[3]]
    
    # Finding the column names for batting table
    batting_columns=[]
    for tag in batting_tables[0].find('thead').find_all('th'):
        batting_columns.append(tag.text)
    batting_columns[0]='Team'
    
    # Find values for the batting table team1
    batting_values_team1=[]
    batting_values_team1.append(batting_tables[0].find('tfoot').find('th').text)
    for tag in batting_tables[0].find('tfoot').find_all('td'):
        batting_values_team1.append(tag.text)
    batting_values_team1[0]=team1
    
    # Find values for the batting table team2
    batting_values_team2=[]
    batting_values_team2.append(batting_tables[1].find('tfoot').find('th').text)
    for tag in batting_tables[1].find('tfoot').find_all('td'):
        batting_values_team2.append(tag.text)
    batting_values_team2[0]=team2
    
    # making batting dataframe
    batting_row_team1=dict(list(zip(batting_columns,batting_values_team1)))
    batting_row_team2=dict(list(zip(batting_columns,batting_values_team2)))
    batting_rows=[batting_row_team1,batting_row_team2]
    batting_df=pd.DataFrame(batting_rows)
    
    # Now pitching tables
    # Finding the column names for pitching table
    pitching_columns=[]
    for tag in pitching_tables[0].find('thead').find_all('th'):
        pitching_columns.append(tag.text)
    pitching_columns[0]='Team'
    
    # Find values for the pitching table team1
    pitching_values_team1=[]
    pitching_values_team1.append(pitching_tables[0].find('tfoot').find('th').text)
    for tag in pitching_tables[0].find('tfoot').find_all('td'):
        pitching_values_team1.append(tag.text)
    pitching_values_team1[0]=team1
    
    # Find values for the pitching table team2
    pitching_values_team2=[]
    pitching_values_team2.append(pitching_tables[1].find('tfoot').find('th').text)
    for tag in pitching_tables[1].find('tfoot').find_all('td'):
        pitching_values_team2.append(tag.text)
    pitching_values_team2[0]=team2
    
    # making pitching dataframe
    pitching_row_team1=dict(list(zip(pitching_columns,pitching_values_team1)))
    pitching_row_team2=dict(list(zip(pitching_columns,pitching_values_team2)))
    pitching_rows=[pitching_row_team1,pitching_row_team2]
    pitching_df=pd.DataFrame(pitching_rows)
    
    # Add homeruns hit to batting_df
    team1_hr_batting=pitching_df['HR'][1]
    team2_hr_batting=pitching_df['HR'][0]

    batting_df.insert(4,'HR_b',[team1_hr_batting,team2_hr_batting])
    
    # get runners in scoring position average
    team1_risp_string=soup.find('div',{'id':'teamrispvisitor'}).text
    team2_risp_string=soup.find('div',{'id':'teamrisphome'}).text

    team1_risp_list=re.findall(r'\d+', team1_risp_string)
    team2_risp_list=re.findall(r'\d+', team2_risp_string)

    team1_risp_hits=int(team1_risp_list[0])
    team2_risp_hits=int(team2_risp_list[0])

    batting_df.insert(4,'H_RISP',[team1_risp_hits,team2_risp_hits])


    try:
        team1_risp_average=int(team1_risp_list[0])/int(team1_risp_list[1])
    except ZeroDivisionError:
        team1_risp_average=0


    try:
        team2_risp_average=int(team2_risp_list[0])/int(team2_risp_list[1])
    except ZeroDivisionError:
        team2_risp_average=0
    
    batting_df.insert(4,'BA_RISP',[team1_risp_average,team2_risp_average])
    
    # get team left on base numbers
    team1_lob_string=soup.find('div',{'id':'teamlobvisitor'}).text
    team2_lob_string=soup.find('div',{'id':'teamlobhome'}).text

    team1_lob_list=re.findall(r'\d+', team1_lob_string)
    team2_lob_list=re.findall(r'\d+', team2_lob_string)

    team1_lob=int(team1_lob_list[0])
    team2_lob=int(team2_lob_list[0])

    batting_df.insert(6,'LOB',[team1_lob,team2_lob])
    
    # Figure out who winner was
    team1_score=int(scorebox_df['R'][0])
    team2_score=int(scorebox_df['R'][1])

    if team1_score>team2_score:
        team1_won=1
        team2_won=0
    
        team1_lost=0
        team2_lost=1

    
    
    else:
        team1_won=0
        team2_won=1
    
        team1_lost=1
        team2_lost=0
        
    # Add the winner column to scorebox_df
    scorebox_df.insert(len(scorebox_df.columns),'Won',[team1_won,team2_won])
    scorebox_df.insert(len(scorebox_df.columns),'Lost',[team1_lost,team2_lost])
    
    batting_df.rename(columns={'R':'R_b','H':'H_b','BB':'BB_b','SO':'SO_b','Pit':'Pit_b','Str':'Str_b','WPA':'WPA_b','aLI':'aLI_b','cWPA':'cWPA_b','acLI':'acLI_b','RE24':'RE24_b'},inplace=True)
    pitching_df.rename(columns={'R':'R_p','H':'H_p','BB':'BB_p','SO':'SO_p','HR':'HR_p','Pit':'Pit_p','Str':'Str_p','WPA':'WPA_p','aLI':'aLI_p','cWPA':'cWPA_p','acLI':'acLI_p','RE24':'RE24_p'},inplace=True)
    
    batting_plus_pitching_df=batting_df.merge(pitching_df,on='Team')
    
    # add errors, won, and lost to batting_plus_pitching_df
    final_df=batting_plus_pitching_df.merge(scorebox_df[['Team','E','Won','Lost']],on='Team')
    
    # find date of the game
    date=soup.find('div',{'class':'scorebox_meta'}).next.next.text
    print('Date: '+date)
    
    # add date to final_df
    final_df['Date']=date
    
    # edit the columns that involve percentages
    # remove the percentages from the inputs
    # change column name to include % symbol

    final_df['WPA-']=final_df['WPA-'].str.replace('%','')
    final_df['cWPA_b']=final_df['cWPA_b'].str.replace('%','')
    final_df['cWPA_p']=final_df['cWPA_p'].str.replace('%','')

    final_df.rename(columns = {'WPA-':'WPA-%', 'cWPA_b':'cWPA_b%','cWPA_p':'cWPA_p%'}, inplace = True)
    
    
    # check for empty values IR and IS
    if final_df['IR'][0]=='':
        final_df.at[0,'IR']='0'
        
    if final_df['IR'][1]=='':
        final_df.at[1,'IR']='0'
        
    if final_df['IS'][0]=='':
        final_df.at[0,'IS']='0'
        
    if final_df['IS'][1]=='':
        final_df.at[1,'IS']='0'    
    
    if link=='https://www.baseball-reference.com/boxes/ANA/ANA202205260.shtml':
        final_df.at[1,'PO']='27'
    
    if link=='https://www.baseball-reference.com/boxes/SEA/SEA202206160.shtml':
        final_df.at[0,'PO']='27'
        
    if link=='https://www.baseball-reference.com/boxes/ATL/ATL202207220.shtml':
        final_df.at[0,'PO']='25'
        
    if link=='https://www.baseball-reference.com/boxes/OAK/OAK202208090.shtml':
        final_df.at[0,'PO']='28'
        
    if link=='https://www.baseball-reference.com/boxes/TOR/TOR202208270.shtml':
        final_df.at[0,'PO']='28'
        
    if link=='https://www.baseball-reference.com/boxes/ANA/ANA202209170.shtml':
        final_df.at[1,'PO']='28'
    
    
    # now convert the dataframe columns to right datatypes


    final_df = final_df.astype({'AB':int,
                                'R_b':int,
                                'H_b':int,
                                'HR_b':int,
                                'RBI':int,
                                'BB_b':int,
                                'SO_b':int,
                                'PA':int,
                                'BA':float,
                                'OBP':float,
                                'SLG':float,
                                'OPS':float,
                                'Pit_b':int,
                                'Str_b':int,
                                'WPA_b':float,
                                'aLI_b':float,
                                'WPA+':float,
                                'WPA-%':float,
                                'cWPA_b%':float,
                                'acLI_b':float,
                                'RE24_b':float,
                                'PO':int,
                                'A':int,
                                'IP':float,
                                'H_p':int,
                                'R_p':int,
                                'ER':int,
                                'BB_p':int,
                                'SO_p':int,
                                'HR_p':int,
                                'ERA':float,
                                'BF':int,
                                'Pit_p':int,
                                'Str_p':int,
                                'Ctct':int,
                                'StS':int,
                                'StL':int,
                                'GB':int,
                                'FB':int,
                                'LD':int,
                                'Unk':int,
                                'GSc':int,
                                'IR':int,
                                'IS':int,
                                'WPA_p':float,
                                'aLI_p':float,
                                'cWPA_p%':float,
                                'acLI_p':float,
                                'RE24_p':float,
                                'E':int,
                                'Won':int,
                                'Lost':int,})

    final_df['Date']=pd.to_datetime(final_df['Date'])
    
    # rearrange some column orders
    # want date, team, won, lost, then the rest


    final_df.insert(0, 'Date', final_df.pop('Date'))
    final_df.insert(2, 'Won', final_df.pop('Won'))
    final_df.insert(3, 'Lost', final_df.pop('Lost'))
    
    # Add dataframe to end of the main one
    main_df=pd.concat([main_df,final_df])
    
    # Pickle the dataframe for backup
    saved_main_df=main_df.to_pickle('saved_main_df.pkl')
    print('Success')
    print('')
    
# Save the dataframe to sql database
main_df.to_sql('2022',engine,index=False)
print('Table saved to database')
    

Link: https://www.baseball-reference.com/boxes/BAL/BAL202209220.shtml
Houston Astros vs Baltimore Orioles
Date: Thursday, September 22, 2022
Success

Link: https://www.baseball-reference.com/boxes/CHA/CHA202209220.shtml
Cleveland Guardians vs Chicago White Sox
Date: Thursday, September 22, 2022
Success

Link: https://www.baseball-reference.com/boxes/CIN/CIN202209220.shtml
Milwaukee Brewers vs Cincinnati Reds
Date: Thursday, September 22, 2022
Success

Link: https://www.baseball-reference.com/boxes/COL/COL202209220.shtml
San Francisco Giants vs Colorado Rockies
Date: Thursday, September 22, 2022
Success

Link: https://www.baseball-reference.com/boxes/KCA/KCA202209220.shtml
Minnesota Twins vs Kansas City Royals
Date: Thursday, September 22, 2022
Success

Link: https://www.baseball-reference.com/boxes/LAN/LAN202209220.shtml
Arizona Diamondbacks vs Los Angeles Dodgers
Date: Thursday, September 22, 2022
Success

Link: https://www.baseball-reference.com/boxes/NYA/NYA202209220.shtml
Boston Re

Atlanta Braves vs Philadelphia Phillies
Date: Sunday, September 25, 2022
Success

Link: https://www.baseball-reference.com/boxes/PIT/PIT202209250.shtml
Chicago Cubs vs Pittsburgh Pirates
Date: Sunday, September 25, 2022
Success

Link: https://www.baseball-reference.com/boxes/TBA/TBA202209250.shtml
Toronto Blue Jays vs Tampa Bay Rays
Date: Sunday, September 25, 2022
Success

Link: https://www.baseball-reference.com/boxes/TEX/TEX202209250.shtml
Cleveland Guardians vs Texas Rangers
Date: Sunday, September 25, 2022
Success

Link: https://www.baseball-reference.com/boxes/BOS/BOS202209260.shtml
Baltimore Orioles vs Boston Red Sox
Date: Monday, September 26, 2022
Success

Link: https://www.baseball-reference.com/boxes/PIT/PIT202209260.shtml
Cincinnati Reds vs Pittsburgh Pirates
Date: Monday, September 26, 2022
Success

Link: https://www.baseball-reference.com/boxes/TOR/TOR202209260.shtml
New York Yankees vs Toronto Blue Jays
Date: Monday, September 26, 2022
Success

Link: https://www.baseball

Miami Marlins vs Milwaukee Brewers
Date: Friday, September 30, 2022
Success

Link: https://www.baseball-reference.com/boxes/NYA/NYA202209300.shtml
Baltimore Orioles vs New York Yankees
Date: Friday, September 30, 2022
Success

Link: https://www.baseball-reference.com/boxes/SDN/SDN202209300.shtml
Chicago White Sox vs San Diego Padres
Date: Friday, September 30, 2022
Success

Link: https://www.baseball-reference.com/boxes/SEA/SEA202209300.shtml
Oakland Athletics vs Seattle Mariners
Date: Friday, September 30, 2022
Success

Link: https://www.baseball-reference.com/boxes/SFN/SFN202209300.shtml
Arizona Diamondbacks vs San Francisco Giants
Date: Friday, September 30, 2022
Success

Link: https://www.baseball-reference.com/boxes/SLN/SLN202209300.shtml
Pittsburgh Pirates vs St. Louis Cardinals
Date: Friday, September 30, 2022
Success

Link: https://www.baseball-reference.com/boxes/TOR/TOR202209300.shtml
Boston Red Sox vs Toronto Blue Jays
Date: Friday, September 30, 2022
Success

Link: https://

In [7]:
pd.set_option('display.max_columns', None)

In [12]:
len(links)

2368

In [13]:
links.index('https://www.baseball-reference.com/boxes/WAS/WAS202210012.shtml')

2367

In [8]:
2368

2368

In [14]:
links[2368]

IndexError: list index out of range